<a href="https://colab.research.google.com/github/pa2e37/dap-2024/blob/main/les06/rep6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


def load_dataset():
    return datasets.load_diabetes()


def describe_dataset(diabetes):
    print("Описание набора данных (DESCR)")
    print("\n".join(diabetes.DESCR.split('\n')[:15]))
    print("... (полное описание доступно в diabetes.DESCR)\n")

    print("Имена признаков")
    print(diabetes.feature_names, "\n")


def create_dataframe(diabetes):
    df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
    df["target"] = diabetes.target

    print("Первые 5 строк DataFrame")
    print(df.head(), "\n")

    return df


def data_types_info(df):
    print("Информация о типах данных")
    print(df.dtypes, "\n")

    print("Количество уникальных значений")
    nunique = df.nunique()
    print(nunique, "\n")

    categorical_like = nunique[nunique <= 10]
    if categorical_like.empty:
        print("Нет признаков, похожих на категориальные.\n")
    else:
        print("Признаки, похожие на категориальные:")
        print(categorical_like, "\n")

    print("=== Пропуски ===")
    print(df.isnull().sum(), "\n")


def correlation_matrix(df, save_fig=False, fig_name='corr_matrix.png'):
    corr = df.corr()

    print("Матрица корреляций (округлённая)")
    print(corr.round(3), "\n")

    plt.figure(figsize=(10, 8))
    plt.title("Матрица корреляций (heatmap)")
    plt.imshow(corr, cmap='bwr', interpolation='none', aspect='auto')
    plt.colorbar()

    ticks = np.arange(len(corr.columns))
    plt.xticks(ticks, corr.columns, rotation=90)
    plt.yticks(ticks, corr.columns)

    plt.tight_layout()

    if save_fig:
        plt.savefig(fig_name, dpi=150)
        print(f"Heatmap сохранён в файл: {fig_name}")

    plt.show()
    return corr


def plot_best_scatter(df, corr):
    target_corr = corr["target"].drop("target")
    best_feature = target_corr.abs().idxmax()
    best_value = target_corr.loc[best_feature]

    print(
        f"Признак с наибольшей корреляцией с target: '{best_feature}' "
        f"(corr = {best_value:.3f})\n"
    )

    plt.figure(figsize=(7, 5))
    plt.scatter(df[best_feature], df["target"], alpha=0.7)
    plt.xlabel(best_feature)
    plt.ylabel("target")
    plt.title(f"target vs {best_feature}\ncorrelation = {best_value:.3f}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return best_feature, best_value


def select_top_k_features(df, corr, k=5):
    target_corr = corr["target"].drop("target")
    topk = target_corr.abs().nlargest(k).index.tolist()

    print(f"Топ-{k} признаков: {topk}\n")

    X = df[topk].copy()
    y = df["target"]
    return X, y, topk


def split_data(X, y, test_size=0.25, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    print(f"Размеры: X_train={X_train.shape}, X_test={X_test.shape}\n")
    return X_train, X_test, y_train, y_test


def train_model(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)

    print("Модель LinearRegression обучена.")
    print("Коэффициенты:", model.coef_)
    print("Смещение (intercept):", model.intercept_, "\n")

    return model


def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print("=== Оценка модели на тестовой выборке ===")
    print(f"RMSE = {rmse:.4f}")
    print(f"R^2  = {r2:.4f}\n")

    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_pred, alpha=0.7)
    plt.plot(
        [y_test.min(), y_test.max()],
        [y_test.min(), y_test.max()],
        linestyle='--'
    )
    plt.xlabel("y — реальные")
    plt.ylabel("y — предсказанные")
    plt.title("Реальные значения vs Предсказанные")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return rmse, r2


def main():
    diabetes = load_dataset()

    describe_dataset(diabetes)
    df = create_dataframe(diabetes)

    data_types_info(df)
    corr = correlation_matrix(df)

    best_feature, best_corr = plot_best_scatter(df, corr)
    X, y, top5 = select_top_k_features(df, corr, k=5)

    X_train, X_test, y_train, y_test = split_data(
        X, y, test_size=0.25, random_state=42
    )

    model = train_model(X_train, y_train)
    rmse, r2 = evaluate_model(model, X_test, y_test)

    return {
        "model": model,
        "top5_features": top5,
        "rmse": rmse,
        "r2": r2,
        "best_feature": best_feature,
        "best_feature_corr": best_corr,
    }


if __name__ == "__main__":
    results = main()


Ответы на вопросы

1. Какие существуют типы машинного обучения?

Основные направления ML:

Обучение с учителем (Supervised Learning)

Классификация

Регрессия

Обучение без учителя (Unsupervised Learning)

Кластеризация

Уменьшение размерности

Ассоциативные правила

Полуруководимое обучение (Semi-supervised Learning)

Обучение с подкреплением (Reinforcement Learning)

Самостоятельное обучение (Self-supervised Learning)

2. В чём различие между обучением с учителем и без учителя?
Параметр	      С учителем	                      Без учителя
Метки	          Присутствуют	                    Отсутствуют
Задача	        Предсказывать целевые значения	  Извлекать скрытые структуры
Типовые задачи  Классификация, регрессия	     Кластеризация, выявление аномалий
Оценка качества	Сравнение с реальными метками	    Внутренние метрики качества
Алгоритмы	      SVM, линейная регрессия	          K-means, PCA

3. Чем отличается пакетное обучение от потокового?
Параметр	            Пакетное обучение	   Динамическое обучение
Как обучается модель	На полном датасете	 По частям, по мере поступления данных
Использование памяти  Требуется весь набор данных	Старые данные можно не хранить
Скорость	            Обучение длительное	 Обновление параметров быстрое
Применение	          Стационарные данные Потоковые или изменяющиеся данные
Примеры	              Традиционные ML-модели	    Online Learning-подходы

4. Отличие обучения на основе примеров от модельного обучения

Instance-based (на основе примеров):

Модель запоминает обучающие записи

Не формирует абстрактной модели

Предсказание делается по похожим точкам

Пример: k-NN

Model-based (модельное):

Строится явная модель с параметрами

Закономерности обобщаются

Обучающие данные после обучения не обязательны

Примеры: линейные модели, деревья решений

5. Что представляет собой линейная регрессия?

Линейная регрессия — это метод, который предсказывает числовой показатель, предполагая линейную зависимость от входных признаков.

Уравнение:

y = B1 + B1X1 + ... + BnXn + e


Задача — подобрать параметры B, минимизирующие ошибку предсказаний.

6. Что такое градиентный спуск?

Градиентный спуск — метод оптимизации, который ищет минимум функции стоимости.

Алгоритм:

Инициализировать параметры случайно

Найти градиент функции ошибки

Двигаться в сторону, противоположную градиенту

Повторять, пока функция не перестанет уменьшаться

Обновление параметров:

O = O - n * Vj(O)

7. Как корректно преобразовывать категориальные признаки?

Популярные способы:

One-Hot Encoding — бинарные столбцы для каждой категории

Label Encoding — присвоение индексов категориям

Ordinal Encoding — упорядочивание категорий с естественным порядком

Target Encoding — замена категорий средним значением целевой переменной

Frequency Encoding — кодирование частотой встречаемости

Советы:

Линейным моделям — One-Hot Encoding

Деревьям решений — Label Encoding допустим

Избегать произвольного порядка категорий

8. Что такое матрица корреляции?

Это таблица, показывающая, насколько линейно связаны пары признаков.

Свойства коэффициента корреляции:

Диапазон: от -1 до 1

1 — сильная прямая связь

-1 — сильная обратная связь

0 — линейной зависимости нет

Назначение:

Поиск мультиколлинеарности

Отбор значимых признаков

Анализ взаимосвязей

9. Что измеряет RMSE?

RMSE — корень из средней квадратичной ошибки: среднее отклонение предсказаний от реальных значений.

Формула:

RMSE = sqrt(E(y_i - Y_i)^2 / n)


Характеристики:

Единицы измерения совпадают с целевой переменной

Сильно штрафует большие ошибки

Уязвим к выбросам

Меньше значение — лучше модель

10. Что показывает R²?

R^2 — доля вариации целевой переменной, объясняемая моделью.

R^2 = 1 - (SS_res / SS_tot)


Интерпретация:

Значения могут быть от –inf до 1

1 — идеальное соответствие

0 — модель не лучше среднего

Отрицательные значения — модель проваливает задачу

Особенность: при добавлении новых признаков R^2 обычно увеличивается, даже если они бесполезны.